# **PPD - Escalabilidade da Multiplicação de Matrizes**

**Caio Ueda Sampaio, 802215, BCC <br>
Vinícius de Oliveira Guimarães, 802431, BCC**

As análises e explicações sobre o código estão no documento PDF enviado.

## Paralelização do primeiro for

Para executar a paralelização considerando o primeiro for, basta executar os códigos que estão no tópico do primeiro for.

In [ ]:
%%writefile mm.c

#include <math.h>
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <unistd.h>
#include <time.h>
#include <omp.h>

int main(int argc, char *argv[]) {
    int lin_a, col_a, lin_b, col_b, lin_c, col_c;
    int i, j, k;

    // Entrada das dimensões das matrizes
    printf("Linhas A: ");   scanf("%d", &lin_a);
    printf("Colunas A / Linhas B: "); scanf("%d", &col_a);
    lin_b = col_a;
    printf("Colunas B: ");  scanf("%d", &col_b);
    printf("\n");
    lin_c = lin_a;
    col_c = col_b;

    // Alocação dinâmica das matrizes
    float *A = (float *) malloc(lin_a * col_a * sizeof(float));
    float *B = (float *) malloc(lin_b * col_b * sizeof(float));
    float *C = (float *) malloc(lin_c * col_c * sizeof(float));

    // Paralelização da inicialização da matriz A
    #pragma omp parallel for private(i)
    for(i = 0; i < lin_a * col_a; i++) {
        unsigned int seed = (unsigned int)time(NULL) ^ (unsigned int) omp_get_thread_num();
        A[i] = (float)rand_r(&seed) / (float)RAND_MAX;
    }

    // Paralelização da inicialização da matriz B
    #pragma omp parallel for private(i)
    for(i = 0; i < lin_b * col_b; i++) {
        unsigned int seed = (unsigned int)time(NULL) ^ (unsigned int) omp_get_thread_num();
        B[i] = (float)rand_r(&seed) / (float)RAND_MAX;
    }

    // Paralelização considerando o primeiro for
    #pragma omp parallel for private(i, j, k)
    for(int i = 0; i < lin_c; i++) {
        for(j = 0; j < col_c; j++) {
            float auxC = 0;
            for(k = 0; k < col_a; k++) {
                auxC += A[i*col_a+k]* B[k*col_b+j];
            }
            C[i*col_c+j] = auxC;
        }
    }

    // Liberação da memória
    free(A);
    free(B);
    free(C);

    return 0;
}

Writing mm.c


In [ ]:
%%writefile script.sh

echo "2048 2048 2048" > 2048

# Compilando!
gcc -Wno-unused-result mm.c -o mm -O3 -fopenmp

for i in {1,2,4,8,16,32,64,128}; do
        for j in {1..3}; do
                echo "$i thread(s) / run $j"
                OMP_NUM_THREADS=$i time ./mm < 2048 > /dev/null
        done
        echo
done

Overwriting script.sh


In [ ]:
!chmod +x script.sh

In [ ]:
!sudo apt install time

In [ ]:
!./script.sh

## Paralelização do segundo for

Para executar a paralelização considerando o segundo for, basta executar os códigos que estão no tópico do segundo for.

In [ ]:
%%writefile mm2.c

#include <math.h>
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <unistd.h>
#include <time.h>
#include <omp.h>

int main(int argc, char *argv[]) {
    int lin_a, col_a, lin_b, col_b, lin_c, col_c;
    int i, j, k;

    // Entrada das dimensões das matrizes
    printf("Linhas A: ");   scanf("%d", &lin_a);
    printf("Colunas A / Linhas B: "); scanf("%d", &col_a);
    lin_b = col_a;
    printf("Colunas B: ");  scanf("%d", &col_b);
    printf("\n");
    lin_c = lin_a;
    col_c = col_b;

    // Alocação dinâmica das matrizes
    float *A = (float *) malloc(lin_a * col_a * sizeof(float));
    float *B = (float *) malloc(lin_b * col_b * sizeof(float));
    float *C = (float *) malloc(lin_c * col_c * sizeof(float));

    // Paralelização da inicialização da matriz A
    #pragma omp parallel for private(i)
    for(i = 0; i < lin_a * col_a; i++) {
        unsigned int seed = (unsigned int)time(NULL) ^ (unsigned int)omp_get_thread_num();
        A[i] = (float)rand_r(&seed) / (float)RAND_MAX;
    }

    // Paralelização da inicialização da matriz B
    #pragma omp parallel for private(i)
    for(i = 0; i < lin_b * col_b; i++) {
        unsigned int seed = (unsigned int)time(NULL) ^ (unsigned int)omp_get_thread_num();
        B[i] = (float)rand_r(&seed) / (float)RAND_MAX;
    }

    // Cálculo da multiplicação de matrizes
    for(int i = 0; i < lin_c; i++) {
        #pragma omp parallel for private(j, k)
        for(j = 0; j < col_c; j++) {
            float auxC = 0;
            for(k = 0; k < col_a; k++) {
                auxC += A[i * col_a + k] * B[k * col_b + j];
            }
            C[i * col_c + j] = auxC;
        }
    }

    // Liberação da memória
    free(A);
    free(B);
    free(C);

    return 0;
}


Overwriting mm2.c


In [ ]:
%%writefile script2.sh

echo "2048 2048 2048" > 2048

# Compilando!
gcc -Wno-unused-result mm2.c -o mm2 -O3 -fopenmp

for i in {1,2,4,8,16,32,64,128}; do
        for j in {1..3}; do
                echo "$i thread(s) / run $j"
                OMP_NUM_THREADS=$i time ./mm2 < 2048 > /dev/null
        done
        echo
done

Overwriting script2.sh


In [ ]:
!chmod +x script2.sh

In [ ]:
!sudo apt install time

In [ ]:
!./script2.sh

## Paralelização terceiro for

Para executar a paralelização considerando o terceiro for, basta executar os códigos que estão no tópico do terceiro for.

In [ ]:
%%writefile mm3.c

#include <math.h>
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <unistd.h>
#include <time.h>
#include <omp.h>

int main(int argc, char *argv[]) {
    int lin_a, col_a, lin_b, col_b, lin_c, col_c;
    int i, j, k;

    // Entrada das dimensões das matrizes
    printf("Linhas A: ");   scanf("%d", &lin_a);
    printf("Colunas A / Linhas B: "); scanf("%d", &col_a);
    lin_b = col_a;
    printf("Colunas B: ");  scanf("%d", &col_b);
    printf("\n");
    lin_c = lin_a;
    col_c = col_b;

    // Alocação dinâmica das matrizes
    float *A = (float *) malloc(lin_a * col_a * sizeof(float));
    float *B = (float *) malloc(lin_b * col_b * sizeof(float));
    float *C = (float *) malloc(lin_c * col_c * sizeof(float));

    // Paralelização da inicialização da matriz A
    #pragma omp parallel for private(i)
    for(i = 0; i < lin_a * col_a; i++) {
        unsigned int seed = (unsigned int)time(NULL) ^ (unsigned int)omp_get_thread_num();
        A[i] = (float)rand_r(&seed) / (float)RAND_MAX;
    }

    // Paralelização da inicialização da matriz B
    #pragma omp parallel for private(i)
    for(i = 0; i < lin_b * col_b; i++) {
        unsigned int seed = (unsigned int)time(NULL) ^ (unsigned int)omp_get_thread_num();
        B[i] = (float)rand_r(&seed) / (float)RAND_MAX;
    }

    // Cálculo da multiplicação de matrizes
    for(int i = 0; i < lin_c; i++) {
        for(int j = 0; j < col_c; j++) {
            float auxC = 0;
            #pragma omp parallel for private(k) reduction(+:auxC)
            for(k = 0; k < col_a; k++) {
                auxC += A[i*col_a+k] * B[k*col_b+ j];
            }
            C[i * col_c + j] = auxC;
        }
    }

    // Liberação da memória
    free(A);
    free(B);
    free(C);

    return 0;
}

Writing mm3.c


In [ ]:
%%writefile script3.sh

echo "2048 2048 2048" > 2048

# Compilando!
gcc -Wno-unused-result mm3.c -o mm3 -O3 -fopenmp

for i in {1,2,4,8,16,32,64,128}; do
        for j in {1..3}; do
                echo "$i thread(s) / run $j"
                OMP_NUM_THREADS=$i time ./mm3 < 2048 > /dev/null
        done
        echo
done

Writing script3.sh


In [ ]:
!chmod +x script3.sh

In [ ]:
!sudo apt install time

In [ ]:
!./script3.sh